In [1]:
import os 

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import pandas as pd

In [2]:
import time
import os 
import numpy as np
import scipy.stats

import sklearn
import sklearn.datasets

import pandas as pd

%matplotlib notebook  
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession

In [3]:
#This is needed to start a Spark session from the notebook
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=100g  pyspark-shell"

from pyspark.sql import SparkSession

In [4]:
#Solution
spark = SparkSession \
    .builder \
    .master("local[2]") \
    .config("spark.driver.maxResultSize", "6G") \
    .appName("task_1_1") \
    .getOrCreate()

22/06/03 21:44:36 WARN Utils: Your hostname, Fatemehs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.178.38 instead (on interface en0)
22/06/03 21:44:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/03 21:44:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/03 21:44:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
#Load the ./data/churn-bigml-80.csv and store in CV_data
train_sessions = spark.read.load('./data/train_sessions.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [6]:
train_purchases = spark.read.load('./data/train_purchases.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [7]:
item_features = spark.read.load('./data/item_features.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [8]:
#adding new column to trian_purchase 
from pyspark.sql.functions import *
train_purchases = train_purchases.withColumn("purchased", lit(1))
#train_purchases.show()

In [9]:
train_sessions = train_sessions.withColumn("purchased", lit(0))

In [10]:
#train_sessions.show()

In [11]:
#concate the train_session with train_purchased
new = train_purchases.union(train_sessions)
#new.show()
#result = df1.union(df2)

In [12]:
new = new.sort("session_id")
#new.show()

In [13]:
#number of items per session

from pyspark.sql import Window
import pyspark.sql.functions as f
w = Window.partitionBy('session_id')
new = new.select('session_id', 'item_id', 'date', 'purchased', f.count('session_id').over(w).alias('number_of_sessions')).sort('session_id', 'item_id')
#new.show()

In [14]:
#number of distinct items per session
number_of_item_per_session = new.groupBy("session_id").agg(countDistinct("item_id").alias("number_of_item_per_session"))
#number_of_item_per_session.show()

In [15]:
#merg two dataframes together
number_of_item_per_session = number_of_item_per_session.withColumnRenamed("session_id","session_id_2")
new = new.join(number_of_item_per_session, new.session_id ==  number_of_item_per_session.session_id_2,"inner") 
new = new.drop("session_id_2")
#new.show()

In [16]:
#number of items in the set for each item_id
w = Window.partitionBy('item_id')
new = new.select('session_id', 'item_id', 'date', 'purchased','number_of_sessions','number_of_item_per_session', f.count('item_id').over(w).alias('number_of_items')).sort('session_id', 'item_id')
#new.show()

In [17]:
#number of session for each item
number_of_session_per_item = new.groupBy("item_id").agg(countDistinct("session_id").alias("number_of_session_per_item"))
number_of_session_per_item = number_of_session_per_item.withColumnRenamed("item_id","item_id_2")
new = new.join(number_of_session_per_item, new.item_id ==  number_of_session_per_item.item_id_2,"inner") 
new = new.drop("item_id_2")
#new.show()

In [18]:
new = new.sort("session_id")
#new.show()

In [19]:
a = item_features.groupBy("item_id").agg(countDistinct("feature_category_id").alias("number_of_categories"), countDistinct("feature_value_id").alias("number_of_values"))
#a.show()

In [20]:
#add this dataframe to the original one so we have the number of features and number of values for each item
a = item_features.groupBy("item_id").agg(countDistinct("feature_category_id").alias("number_of_categories"), countDistinct("feature_value_id").alias("number_of_values"))
a = a.withColumnRenamed("item_id","item_id_2")
new = new.join(a, new.item_id ==  a.item_id_2,"inner") 
new = new.drop("item_id_2")
#new.show()

In [21]:
#join sessions with item_features
new_train = train_purchases.union(train_sessions)
new_train = new_train.drop("date")
new_train = new_train.drop("purchased")
#new_train.show()

In [22]:
#here we add session to the item_features dataframe
new_train = new_train.withColumnRenamed("item_id","item_id_2")
new_train = new_train.join(item_features, new_train.item_id_2 ==  item_features.item_id,"inner") 
new_train = new_train.drop("item_id_2")
#new_train.show()

In [23]:
#we count the number of categories and values for each session
b = new_train.groupBy("session_id").agg(countDistinct("feature_category_id").alias("number_of_categories_for_session"), countDistinct("feature_value_id").alias("number_of_values_for_session"))
#b.show()

In [24]:
#we add these two features to the original dataframe
b = b.withColumnRenamed("session_id","session_id_2")
new = new.join(b, new.session_id ==  b.session_id_2,"inner") 
new = new.drop("session_id_2")
#new.show()


In [25]:
#start playing with the time
#add epoch as a new feature
new = new.withColumn( "epoch_seconds", 
      unix_timestamp(col("date"),"yyyy-MM-dd HH:mm:ss.SSS").alias("date"))
#new.show()

In [26]:
#converting date string to date
new = new.withColumn("time", to_timestamp(new.date, 'yyyy-MM-dd HH:mm:ss.SSS').alias('dt'))
#train_sessions.show()

In [27]:
from pyspark.sql.functions import year, month, dayofweek
new = new.withColumn('dayOfWeek', dayofweek(col('time')))
new = new.withColumn('month', month(col('time')))
new = new.withColumn('year', year(col('time')))

In [28]:
new = new.withColumn('hour', hour(col('time')))
new = new.withColumn('minute', minute(col('time')))
new = new.withColumn('dayofmonth', dayofmonth(col('time')))
new = new.withColumn('dayofyear', dayofyear(col('time')))
new = new.withColumn('weekofyear', weekofyear(col('time')))

In [36]:
new.count()

5743820

In [ ]:
#we want to add category ids to the dataset

In [29]:
item_features = item_features.withColumnRenamed("item_id","item_id_2")

In [30]:
test1 = train_purchases.union(train_sessions)
test1 = test1.drop('date')
test1 = test1.drop('session_id')
test1 = test1.drop('purchased')
#test1.show()

In [31]:
test2 = test1.groupby("item_id").count()

In [32]:
test2.count()

23618

In [33]:
test2 = test2.withColumnRenamed("item_id","item_id_2")

In [34]:
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import * 

In [30]:
#we add category ids as features to our dataframe
for i in range(1,74):
    #print(i)
    cond = (test1.item_id == item_features.item_id_2)
    test1 = test1.join(item_features, cond, how="inner").withColumn("category_id: " + str(i), when((col("feature_category_id")==i), col("feature_value_id")).otherwise(0))
    test1 = test1.drop("item_id_2")
    test1 = test1.drop("feature_category_id")
    test1 = test1.drop("feature_value_id")
    test1 = test1.groupBy("item_id").agg(F.max("category_id: " + str(i)).alias("category_id: " + str(i)))
    test2 = test2.join(test1,test2.item_id_2 == test1.item_id, how = "inner")
    test2 = test2.drop("item_id")
    #print('f')

In [20]:
test2.count()

23618

In [36]:
#we merge the dataframe with the category ids with the other one that has date and new features
new_final = new.join(test2, new.item_id ==  test2.item_id_2,"inner") 

In [37]:
new_final = new_final.drop('date','epoch_seconds','time','dayOfweek',"month",'year','item_id_2','count','purchased')

In [38]:
new_final.toPandas().to_csv('X_update.csv', index=False)

22/06/02 20:38:58 WARN DAGScheduler: Broadcasting large task binary with size 31.5 MiB


In [35]:
new_final.dtypes

[('item_id', 'int'),
 ('number_of_sessions', 'bigint'),
 ('number_of_item_per_session', 'bigint'),
 ('number_of_items', 'bigint'),
 ('number_of_session_per_item', 'bigint'),
 ('number_of_categories', 'bigint'),
 ('number_of_values', 'bigint'),
 ('number_of_categories_for_session', 'bigint'),
 ('number_of_values_for_session', 'bigint'),
 ('category_id: 1', 'int'),
 ('category_id: 2', 'int'),
 ('category_id: 3', 'int'),
 ('category_id: 4', 'int'),
 ('category_id: 5', 'int'),
 ('category_id: 6', 'int'),
 ('category_id: 7', 'int'),
 ('category_id: 8', 'int'),
 ('category_id: 9', 'int'),
 ('category_id: 10', 'int'),
 ('category_id: 11', 'int'),
 ('category_id: 12', 'int'),
 ('category_id: 13', 'int'),
 ('category_id: 14', 'int'),
 ('category_id: 15', 'int'),
 ('category_id: 16', 'int'),
 ('category_id: 17', 'int'),
 ('category_id: 18', 'int'),
 ('category_id: 19', 'int'),
 ('category_id: 20', 'int'),
 ('category_id: 21', 'int'),
 ('category_id: 22', 'int'),
 ('category_id: 23', 'int'),
 ('c

In [50]:
#new_final.count()
print(len(new_final.columns))

#print(new_final.shape())

97


In [34]:
#convert dataframe to RDD

final_rdd = final.rdd

# preparing test set

In [35]:
X = spark.read.load('X.csv', 
                     format='com.databricks.spark.csv', 
                     header='true', 
                     inferSchema='true')

In [36]:
test_final_sessions = spark.read.load('./data/test_final_sessions.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [37]:
test_leaderboard_sessions = spark.read.load('./data/test_leaderboard_sessions.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [44]:
item_features = spark.read.load('./data/item_features.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [45]:
from pyspark.sql import Window
import pyspark.sql.functions as f
w = Window.partitionBy('session_id')

df_1 = test_leaderboard_sessions.select('session_id', 'item_id', 'date', f.count('session_id').over(w).alias('number_of_sessions')).sort('session_id', 'item_id')
#df_1.show()

In [46]:
#number of distinct items per session
from pyspark.sql.functions import * 
df_2 = test_leaderboard_sessions.groupBy("session_id").agg(countDistinct("item_id").alias("number_of_item_per_session"))
#df_2.show()

In [47]:
#merg two dataframes together
df_2 = df_2.withColumnRenamed("session_id","session_id_2")
df_1 = df_1.join(df_2, df_1.session_id ==  df_2.session_id_2,"inner") 
df_1 = df_1.drop("session_id_2")
#df_1.show()

In [48]:
#number of items in the set for each item_id
w = Window.partitionBy('item_id')
df_1 = df_1.select('session_id', 'item_id', 'date','number_of_sessions','number_of_item_per_session', f.count('item_id').over(w).alias('number_of_items')).sort('session_id', 'item_id')
#df_1.show()

In [49]:
#number of session for each item
number_of_session_per_item = df_1.groupBy("item_id").agg(countDistinct("session_id").alias("number_of_session_per_item"))
number_of_session_per_item = number_of_session_per_item.withColumnRenamed("item_id","item_id_2")
df_1 = df_1.join(number_of_session_per_item, df_1.item_id ==  number_of_session_per_item.item_id_2,"inner") 
df_1 = df_1.drop("item_id_2")
#df_1.show()

In [50]:
#add this dataframe to the original one so we have the number of features and number of values for each item
df_3 = item_features.groupBy("item_id").agg(countDistinct("feature_category_id").alias("number_of_categories"), countDistinct("feature_value_id").alias("number_of_values"))
df_3 = df_3.withColumnRenamed("item_id","item_id_2")
df_1 = df_1.join(df_3, df_1.item_id ==  df_3.item_id_2,"inner") 
df_1 = df_1.drop("item_id_2")
#new.show()

In [51]:
#join sessions with item_features
new_test = test_final_sessions.drop("date")
#new_train.show()

In [52]:
#here we add session to the item_features dataframe
new_test = new_test.withColumnRenamed("item_id","item_id_2")
new_test = new_test.join(item_features, new_test.item_id_2 ==  item_features.item_id,"inner") 
new_test = new_test.drop("item_id_2")
#new_train.show()

In [53]:
#we count the number of categories and values for each session
b = new_test.groupBy("session_id").agg(countDistinct("feature_category_id").alias("number_of_categories_for_session"), countDistinct("feature_value_id").alias("number_of_values_for_session"))
#b.show()
#we add these two features to the original dataframe
b = b.withColumnRenamed("session_id","session_id_2")
df_1 = df_1.join(b, df_1.session_id ==  b.session_id_2,"inner") 
df_1 = df_1.drop("session_id_2")
#new.show()

In [54]:
#start playing with the time
#add epoch as a new feature
df_1 = df_1.withColumn( "epoch_seconds", 
      unix_timestamp(col("date"),"yyyy-MM-dd HH:mm:ss.SSS").alias("date"))
#new.show()

In [55]:
#converting date string to date
df_1 = df_1.withColumn("time", to_timestamp(df_1.date, 'yyyy-MM-dd HH:mm:ss.SSS').alias('dt'))
#train_sessions.show()

In [56]:
from pyspark.sql.functions import year, month, dayofweek
df_1 = df_1.withColumn('dayOfWeek', dayofweek(col('time')))
df_1 = df_1.withColumn('month', month(col('time')))
df_1 = df_1.withColumn('year', year(col('time')))

In [57]:
df_1 = df_1.withColumn('hour', hour(col('time')))
df_1 = df_1.withColumn('minute', minute(col('time')))
df_1 = df_1.withColumn('dayofmonth', dayofmonth(col('time')))
df_1 = df_1.withColumn('dayofyear', dayofyear(col('time')))
df_1 = df_1.withColumn('weekofyear', weekofyear(col('time')))

In [58]:
item_features = item_features.withColumnRenamed("item_id","item_id_2")

In [59]:
test1 = test_leaderboard_sessions
test1 = test1.drop('date','session_id')
#test1.show()

In [60]:
test2 = test1.groupby("item_id").count()

In [61]:
test2 = test2.withColumnRenamed("item_id","item_id_2")

In [62]:
#we add category ids as features to our dataframe
for i in range(1,74):
    #print(i)
    cond = (test1.item_id == item_features.item_id_2)
    test1 = test1.join(item_features, cond, how="inner").withColumn("category_id: " + str(i), when((col("feature_category_id")==i), col("feature_value_id")).otherwise(0))
    test1 = test1.drop("item_id_2")
    test1 = test1.drop("feature_category_id")
    test1 = test1.drop("feature_value_id")
    test1 = test1.groupBy("item_id").agg(f.max("category_id: " + str(i)).alias("category_id: " + str(i)))
    test2 = test2.join(test1,test2.item_id_2 == test1.item_id, how = "inner")
    test2 = test2.drop("item_id")
    #print('f')

In [63]:
#we merge the dataframe with the category ids with the other one that has date and new features
final_test = df_1.join(test2, df_1.item_id ==  test2.item_id_2,"inner") 

In [65]:
final_test.dtypes

[('session_id', 'int'),
 ('item_id', 'int'),
 ('date', 'string'),
 ('number_of_sessions', 'bigint'),
 ('number_of_item_per_session', 'bigint'),
 ('number_of_items', 'bigint'),
 ('number_of_session_per_item', 'bigint'),
 ('number_of_categories', 'bigint'),
 ('number_of_values', 'bigint'),
 ('number_of_categories_for_session', 'bigint'),
 ('number_of_values_for_session', 'bigint'),
 ('epoch_seconds', 'bigint'),
 ('time', 'timestamp'),
 ('dayOfWeek', 'int'),
 ('month', 'int'),
 ('year', 'int'),
 ('hour', 'int'),
 ('minute', 'int'),
 ('dayofmonth', 'int'),
 ('dayofyear', 'int'),
 ('weekofyear', 'int'),
 ('item_id_2', 'int'),
 ('count', 'bigint'),
 ('category_id: 1', 'int'),
 ('category_id: 2', 'int'),
 ('category_id: 3', 'int'),
 ('category_id: 4', 'int'),
 ('category_id: 5', 'int'),
 ('category_id: 6', 'int'),
 ('category_id: 7', 'int'),
 ('category_id: 8', 'int'),
 ('category_id: 9', 'int'),
 ('category_id: 10', 'int'),
 ('category_id: 11', 'int'),
 ('category_id: 12', 'int'),
 ('categor

In [68]:
final_test_rdd = final_test.rdd

In [49]:
#final_test1 = final_test.drop('epoch_seconds','time','date','dayOfWeek','month','year','hour','minute','dayofmonth','dayofyear')

In [77]:
#final_test1 = final_test.drop('date','item_id_2','count')

In [66]:
#some2 = final_test1.toPandas()

In [82]:
#some2.to_csv('leaderboard_test.csv', index=False)

In [ ]:
#some.to_csv('final_test.csv', index=False)

In [67]:
#some = final_test1.toPandas()

In [52]:
#some.to_csv('final_test.csv', index=False)